In [1]:
from trendAnalyserPy import StockTrendAnalyzer
from backtestPy import Backtester
from optimizerPy import Optimizer
from stockData import StockData
from strategies import Strategies
import yfinance as yf
import numpy as np
import pandas as pd
import datetime as dt

In [5]:
class Trading:
    def __init__(self, symbol, start_date, end_date, interval, specific_given_date=None, period=None):
        self.symbol = symbol
        self.start_date = start_date
        self.end_date = end_date
        self.interval = interval
        self.period = period
        self.stock_intraday = None

        # Initialize the detector with properly loaded data
        self._load_data(specific_given_date, period)
        self.detector = StockTrendAnalyzer(symbol, start_date, end_date, self.stock, interval)
        self.backtester = Backtester(symbol, start_date, end_date, interval, self.stock)
        self.optimizer = Optimizer(self.backtester)

        self.peaks = []
        self.throughs = []
        self.uptrends = []
        self.downtrends = []
        self.peaks_intraday = []
        self.troughs_intraday = []
        self.uptrends_intraday = []
        self.downtrends_intraday = []
        

    def _load_data(self, specific_given_date, period):
        """
        Load stock data, considering intraday intervals with period and start_date.
        """
        self.load_data = StockData(symbol, start_date, end_date, interval, specific_given_date, period)
        if self.interval in ['1m', '2m', '5m', '15m', '30m', '60m', '90m']: 
            self.stock, self.stock_intraday = self.load_data.load_intraday_data()
        else:
            self.stock = self.load_data.load_daily_data()

    def analyze_stock(self):
        """
        Analyze stock trends for the initialized symbol and time period.
        """
        analyzer = self.detector
        analyzer.data = self.stock  # Ensure detector works with the loaded data
        self.peaks, self.troughs = analyzer.find_peaks_and_troughs()
        self.uptrends, self.downtrends = analyzer.identify_trends()

        # Print trend summary
        print(f"\nTrend Summary for {self.symbol}:")
        for summary in analyzer.get_trend_summary():
            print(summary)

        # Generate visualization
       
        if self.stock_intraday is not None and len(self.stock_intraday) > 0:
            analyzer.data = self.stock_intraday
            self.peaks_intraday, self.troughs_intraday = analyzer.find_peaks_and_troughs()
            self.uptrends_intraday, self.downtrends_intraday = analyzer.identify_trends()
            last_peak_idx = self.peaks[-1]
            last_trough_idx = self.troughs[-1]
            last_daily_peak = self.stock.iloc[last_peak_idx]['High']
            last_daily_trough = self.stock.iloc[last_trough_idx]['Low']
            #analyzer.visualize_intraday_trends(self.stock.iloc[-1], last_daily_peak, last_daily_trough)
            
 
        return analyzer

    def analyze_and_backtest(self, strategy=None, **kwargs):
        """
        Perform analysis to find peaks and troughs, and then backtest using the results.
        """
        # Step 1: Analyze data to find peaks and troughs
        peaks = self.peaks
        troughs = self.troughs
        uptrends = self.uptrends
        downtrends = self.downtrends
    
        # Step 2: Update params with peaks and troughs
        params = {}
        params["peaks"] = peaks
        params["troughs"] = troughs
        params["uptrends"] = uptrends
        params["downtrends"] = downtrends
    
        # Step 3: Initialize Backtester with data and params
        backtester = Backtester(
            symbol=self.symbol,
            start_date=self.start_date,
            end_date=self.end_date,
            interval=self.interval,
            data=self.stock,
            initial_capital=10000,
            strategy=strategy,
            **params
        )
    
        # Step 4: Run the backtest and return results
        results = backtester.backtest(strategy, **kwargs)
        plt = backtester.visualize_data()
        plt.show()
        if results is not None:
            backtester.print_results(results)
        return backtester

    def backtest_intraday(self, analyzer, strategy=None, **kwargs):
        params = {}
        params["peaks"] = self.peaks
        params["troughs"] = self.troughs
        params["uptrends"] = self.uptrends
        params["downtrends"] = self.downtrends
        params["peaks_intraday"] = self.peaks_intraday
        params["troughs_intraday"] = self.troughs_intraday
        params["uptrends_intraday"] = self.uptrends_intraday
        params["downtrends_intraday"] = self.downtrends_intraday
        params["daily_data"] = self.stock

        start_date = self.stock_intraday.index[0].strftime("%Y-%m-%d")
        end_date = self.stock_intraday.index[-1].strftime("%Y-%m-%d")
        backtester = Backtester(
            symbol=self.symbol,
            start_date=start_date,
            end_date=end_date,
            interval=self.interval,
            data=self.stock_intraday,
            initial_capital=10000,
            strategy=strategy,
            **params
        )
        results = backtester.backtest(strategy, **kwargs)

        if results is not None:
            last_peak_idx = self.peaks[-1]
            last_trough_idx = self.troughs[-1]
            last_daily_peak = self.stock.iloc[last_peak_idx]['High']
            last_daily_trough = self.stock.iloc[last_trough_idx]['Low']
            backtester.print_results(results)
            if self.period:
                backtester.get_trades_info_per_day()
            else:
                backtester.visualize_intraday_data(self.stock.iloc[-1],last_daily_peak, last_daily_trough)
        return backtester
        
    def optimize(self, strategy, param_ranges, initial_guess):
        """
        Optimize trading strategy parameters.
        """
        backtester = Backtester(self.stock)
        optimizer = Optimizer(backtester)
        return optimizer.optimize(strategy, param_ranges, initial_guess)


In [9]:
symbol='SPY'
start_date='2024-01-21' 
end_date='2024-06-11'
interval='5m'
period='55d'
specific_given_date='2024-12-11'
trading = Trading(symbol, start_date, end_date, interval=interval, specific_given_date=specific_given_date, period=period)
analyzer = trading.analyze_stock()
strategies = Strategies()
#backtest = trading.analyze_and_backtest(strategy=strategies.gaps_strategy)
#backtest = trading.backtest_intraday(analyzer,strategy=strategies.multi_indicator_strategy, ema_fast=12,
    #ema_slow=26,
    #rsi_period=14,
    #rsi_oversold=30,
    #rsi_overbought=70,
    #macd_fast=12,
    #macd_slow=26,
    #macd_signal=9,
    #bb_period=20,
    #bb_std=2)
backtest = trading.backtest_intraday(analyzer,strategy=strategies.gaps_strategy)

[*********************100%***********************]  1 of 1 completed

*********************100%***********************]  1 of 1 completed


Trend Summary for SPY:
Uptrend: Start: 2024-01-17 (Price: $472.29), End: 2024-04-04 (Price: $513.07), Change: 8.6%
Uptrend: Start: 2024-05-02 (Price: $505.03), End: 2024-05-23 (Price: $525.96), Change: 4.1%
Uptrend: Start: 2024-06-25 (Price: $544.83), End: 2024-07-24 (Price: $541.23), Change: -0.7%
Uptrend: Start: 2024-08-28 (Price: $558.30), End: 2024-09-03 (Price: $552.08), Change: -1.1%
Uptrend: Start: 2024-09-20 (Price: $568.25), End: 2024-10-31 (Price: $568.64), Change: 0.1%
Uptrend: Start: 2024-11-15 (Price: $585.75), End: 2024-12-10 (Price: $602.80), Change: 2.9%
Downtrend: Start: 2024-04-04 (Price: $513.07), End: 2024-04-26 (Price: $508.26), Change: -0.9%
Downtrend: Start: 2024-08-01 (Price: $543.01), End: 2024-08-19 (Price: $559.61), Change: 3.1%

BACKTEST RESULTS
Symbol:                  SPY			Buy & Hold Net Profit:   $ 362.33
Start Date:              2024-10-14		Buy & Hold Profit:     3.62%
End Date:                2024-12-10
------------------------------------------------

In [11]:
backtest.get_trades_info()

,type,entry_price,exit_price,entry_idx,exit_idx,profit_percent,profit_money,open_date,close_date
0,long,584.750000,583.849976,234,235,-0.153916,-15.391610,2024-10-17 09:30:00-04:00,2024-10-17 09:35:00-04:00
1,short,580.950012,580.950012,468,469,-0.000000,-0.000000,2024-10-22 09:30:00-04:00,2024-10-22 09:35:00-04:00
2,short,581.434998,581.340027,546,547,0.016334,1.633385,2024-10-23 09:30:00-04:00,2024-10-23 09:35:00-04:00
3,long,579.109985,579.299927,624,625,0.032799,3.279885,2024-10-24 09:30:00-04:00,2024-10-24 09:35:00-04:00
4,long,582.109985,582.280029,702,703,0.029212,2.921165,2024-10-25 09:30:00-04:00,2024-10-25 09:35:00-04:00
5,long,582.210022,581.869995,780,781,-0.058403,-5.840278,2024-10-28 09:30:00-04:00,2024-10-28 09:35:00-04:00
6,long,593.299988,593.359985,1404,1405,0.010113,1.011252,2024-11-07 09:30:00-05:00,2024-11-07 09:35:00-05:00
7,short,589.219971,589.320007,1872,1873,-0.016978,-1.697781,2024-11-15 09:30:00-05:00,2024-11-15 09:35:00-05:00
8,short,585.000000,585.289978,2028,2029,-0.049569,-4.956889,2024-11-19 09:30:00-05:00,2024-11-19 09:35:00-05:00
9,long,591.650024,589.090027,2184,2185,-0.432688,-43.268781,2024-11-21 09:30:00-05:00,2024-11-21 09:35:00-05:00


In [ ]:
#1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo